In [1]:
import nbimporter

# Import 1d well examples
from GPDM_kwells_simulate import *

plotly.offline.init_notebook_mode(connected=True)

from joblib import Parallel, delayed
import multiprocessing
import itertools

num_cores = multiprocessing.cpu_count()

# %run GPDM_direct_fixedpoints.ipynb
# %run GPDM_Examples.ipynb

Importing Jupyter notebook from GPDM_kwells_simulate.ipynb
Importing Jupyter notebook from GPDM_direct_fixedpoints.ipynb


/nfs/nhome/live/gbohner/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



# Experimental setup

1. We first generate training and test data with the same parameters.

2. We then fit GP models with a fixed number of parameters, but vary the number of fixed points versus basic inducing points

3. We compare the performance of the GP models against one-another as well as against linear autoregression.

## 1. Generate training and test data

In [2]:
kwells_params = OrderedDict()
kwells_params["rseed"] = 1234 # Happens to creates balanced training set in the first 8 trials in y
kwells_params["Ny"] = 500
kwells_params["T"] = 20
kwells_params["slope"] = -0.9
kwells_params["Sigma_eps"] = 5e-2
kwells_params["well_locs"] = np.array([1.0, -1.0])
kwells_params["well_width"] = 3
kwells_params["Sigma_nu"] = 1e-3
kwells_params["mu_0_0"] = 0.
kwells_params["Sigma_0_0"] = 1e-4


# Generate trials and collect them into [D x T x N] array
np.random.seed(kwells_params["rseed"])
all_trials_x = []
all_trials_y = []
for n in range(kwells_params["Ny"]):
    x,y = kwells_draw_trial(transform_func = (lambda x: x**3), **kwells_params)
    
    all_trials_x.append(x[:,:,None])
    all_trials_y.append(y[:,:,None])
    
x = np.concatenate(all_trials_x, axis=2)
y = np.concatenate(all_trials_y, axis=2)

# Swap around trials to ensure balanced training sets
cur_trial = 0
cur_trial_type = 1.
while cur_trial < 0.4*kwells_params["Ny"]:
    # If current trial is not correct type, switch it with one that is
    if not (np.mean(y[:,(kwells_params["T"]-4):,cur_trial]) * cur_trial_type > 0.2):
        switch_trial = cur_trial+1
        while (not (np.mean(y[:,(kwells_params["T"]-4):,switch_trial]) * cur_trial_type > 0.2)):
            switch_trial += 2
            
        tmp = np.copy(y[:,:,switch_trial])
        y[:,:,switch_trial] = np.copy(y[:,:,cur_trial])
        y[:,:,cur_trial] = tmp
        
        tmp = np.copy(x[:,:,switch_trial])
        x[:,:,switch_trial] = np.copy(x[:,:,cur_trial])
        x[:,:,cur_trial] = tmp
        
    #print np.mean(y[:,(kwells_params["T"]-4):,cur_trial]) * cur_trial_type
    cur_trial += 1
    cur_trial_type *= -1.

plots_by_run = []
for v in range(128):
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(kwells_params["T"])), 
                      y=np.squeeze(x[:,:,v]), 
                      mode='lines')
    )
    
    
print rmse(pred_lin_AR1(y[:,:,200:], y[:,:,:50], cutoff=None), y[:,:,200:])
    
    
xstar = np.atleast_2d(np.arange(-2.5,2.5,0.05))
true_tr_vals = kwells_true_tr_fnc(xstar, **kwells_params)

#set_trace()

plt([plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange')),
 plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals+np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange', size=2)),
 plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals-np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange', size=2))
     ])

plt(plots_by_run)

0.365501842329


## 2. Fitting GP models

In [3]:
def fit_GP(num_fixed_points, num_trials, batchnum, start_time):
    # Set number of parameters
    D = 1
    Ns = num_fixed_points
    Nz = 16 - Ns

    # Fix the training data
    y_train = y[:,:,batchnum*num_trials:(batchnum*num_trials+num_trials)]
    x_train = x[:,:,batchnum*num_trials:(batchnum*num_trials+num_trials)]

    np.random.seed(kwells_params['rseed']*2)

    #######################################################
    # Initialise the parameters
    paramdict = init_params(y_train, D, Nz, Ns)

    # Fix noise to true value
    paramdict['Sigma_nu'] = kwells_params['Sigma_nu'] * np.ones_like(paramdict['Sigma_nu'])

#     # Re-initialise / add some more parameters
#     paramdict['Sigma_s'] = 1e-2*np.ones((Ns,1))
#     paramdict['Sigma_J'] = 1e-2*np.ones((Ns*D,1))
    
    # Add transformations for certain parameters 
    # (Note that the parameter indices may be changed by transforms! (for cholesky repres of matrices, "SquareMatrix" type))
    transforms = OrderedDict()
    for par in ['Sigma_0_0', 'Sigma_u', 'Sigma_s', 'Sigma_J', 'lengthscales', 'Sigma_eps', 'Sigma_nu', 'kernel_variance']:
        transforms[par] = {}
        transforms[par]['type'] = "Square"
    
    # Create vectorised and transformed representation
    (init_paramvec, dict_ind, dict_shape) = params_to_vec(paramdict, transforms=transforms)
 
    #######################################################
    # Optimise only certain elements of paramvec (messy with indices)
    opt_params = np.arange(init_paramvec.shape[0])
    # opt_params = np.delete(opt_params, np.hstack([dict_ind['C'], dict_ind['Sigma_nu'], dict_ind['J']])) # All except the ones listed here
    opt_params = np.delete(opt_params, np.hstack([dict_ind['C'], dict_ind['Sigma_nu']])) # All except the ones listed here
    cur_pvec = init_paramvec[opt_params]

    #######################################################
    # Add bounds for parameters 
    bnds = list(((None, None),) * init_paramvec.shape[0])
    # cur_dim = 0
    # cur_z = 0
    # cur_tot = 0
    # z_mins = np.min(z, axis=1)
    # z_maxs = np.max(z, axis=1)
    # for i in np.concatenate([dict_ind['z'], dict_ind['s']]): # Note the idiotic python reshape order for setting bounds per dim
    #     z_min = z_mins[cur_dim]
    #     z_max = z_maxs[cur_dim]
    #     bnds[i] = (z_min-0.05*(z_max-z_min), z_max+0.05*(z_max-z_min))
    #     cur_z = cur_z+1
    #     if cur_tot < D*Nz:
    #         cur_z = np.mod(cur_z, Nz)
    #     else:
    #         cur_z = np.mod(cur_z, Ns)
    #     cur_tot = cur_tot+1
    #     if cur_z==0:
    #         cur_dim = cur_dim+1
    #     if cur_tot==D*Nz:
    #         cur_dim = 0
    # for i in np.concatenate([dict_ind['J']]):
    #     bnds[i] = (-1., 1.)
    bnds_final = []
    for i in opt_params:
        bnds_final.append(bnds[i])
    bnds = tuple(bnds_final)

    #######################################################
    # Add priors (to span at least the bounds)
    priors = []
    
    # Add prior to ensure inducing points are smooth
    cur_prior = {}
    cur_prior['type'] = "InducingSmooth_and_DPP"
    cur_prior['metadata'] = {}
    def unpack_dict_tmp(pdict):
        kernelparams = {'lengthscales': pdict['lengthscales'], 'kernel_variance': pdict['kernel_variance']}
        # Return the parameters we want in the required format (joint smoothness of (z-u) and s)
        return (pdict['u'], 
                pdict['z'],
                pdict['Sigma_u'],
                kernelparams)
    cur_prior['metadata']['unpack_dict'] = unpack_dict_tmp
    cur_prior['metadata']['kernel_func'] = RBF
    cur_prior['metadata']['prior_weight_Smooth'] = 1e0
    cur_prior['metadata']['prior_weight_DPP'] = 1e0
    priors.append(cur_prior)
    
    # Add prior to ensure fixed points are smooth
    cur_prior = {}
    cur_prior['type'] = "InducingSmooth_and_DPP"
    cur_prior['metadata'] = {}
    def unpack_dict_tmp(pdict):
        kernelparams = {'lengthscales': pdict['lengthscales'], 'kernel_variance': pdict['kernel_variance']}
        # Return the parameters we want in the required format (joint smoothness of (z-u) and s)
        return (pdict['s'], 
                pdict['s'],
                pdict['Sigma_s'],
                kernelparams)
    cur_prior['metadata']['unpack_dict'] = unpack_dict_tmp
    cur_prior['metadata']['kernel_func'] = RBF
    cur_prior['metadata']['prior_weight_Smooth'] = 1e0
    cur_prior['metadata']['prior_weight_DPP'] = 1e0
    priors.append(cur_prior)

    # # Add a strong prior to learn actual fixed points
    # logGamma_prior = create_prior("LogGamma", [2., 0.5, -6.])
    # for i in np.concatenate([dict_ind['Sigma_s'], dict_ind['Sigma_J']]):
    #     prior_funcs[i] = logGamma_prior

    # tmp_x = np.logspace(-6.0,2,100)    
    # plt(plt_type.Figure(data=[plt_type.Scatter(x=tmp_x, y=np.exp(-logGamma_prior(tmp_x)))], layout=plt_type.Layout(xaxis=dict(type= "log"))))
    # plt(plt_type.Figure(data=[plt_type.Scatter(x=tmp_x, y=logGamma_prior(tmp_x))], layout=plt_type.Layout(xaxis=dict(type= "log"))))

    #######################################################
    # Prepare the optimisation
#     tmp_func = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
#                                                 y_train, dict_ind, dict_shape, 
#                                                 transforms=transforms,
#                                                 priors=priors)[0])
#     objective_with_grad = value_and_grad(tmp_func, argnum=0)

    # Prepare the optimisation
    f_objective = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                                y_train, dict_ind, dict_shape, 
                                                transforms=transforms,
                                                priors=priors)[0])
    
    def minibatch_sample_func(data):
        a = random.sample(range(data.shape[2]/2),1)[0]
        return data[:,:,[2*a, 2*a+1]]
        
    f_minibatch = lambda pvec_partial: (minibatch_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                                y_train, dict_ind, dict_shape, 
                                                transforms=transforms,
                                                priors=priors,
                                                minibatch_func=minibatch_sample_func)[0])
    f_minibatch_with_grad = grad(f_minibatch, argnum=0)


    # By iterating minimize within a for cycle, we can save all intermediate results and set ending times
    save_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (num_fixed_points, num_trials, batchnum)
    save_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + save_fname_params + ".pkl"
    init_time = time.time()
    max_time = 12.0*3600 # Maximum iteration time in seconds, break if reached
    all_results = []

#     for it in range(100):
#         result = scipy.optimize.minimize(objective_with_grad, cur_pvec, jac=True, method='L-BFGS-B', bounds=bnds, callback=None,
#                               options={'maxiter':10, 'disp':True})
    for it in range(1):
        np.random.seed(1234+it)
        result = adamOptimize(f_objective, f_minibatch_with_grad, cur_pvec,
                          options={'maxiter':300, 'disp':True})
#                             callback_func=kwells_callback_plot_external,
#                              callback_params=[opt_params, init_paramvec, transforms, dict_ind, dict_shape,
#                                   kwells_params])
        all_results.append(result)
        # Save the results
        with open(save_fname, 'wb') as f:
            pickle.dump([y_train, x_train, kwells_params,
                         all_results, 
                         init_paramvec, dict_ind, dict_shape, opt_params, 
                         bnds, transforms], f)
        cur_pvec = result.x
        cur_time = time.time()
        #print([it, cur_time - init_time, result.fun])

        # Exit if maximum time is reached

        if ((cur_time - init_time) > max_time):
            print(["Maximum iteration time reached at iter", it])
            break

        if len(all_results)>=2:
            if (all_results[-1].fun - all_results[-2].fun) >= (-1e-2*num_trials):
                print(["Update did not improve objective function, stopping"])
                break

In [4]:
# fit_GP(3, 4, 0, datetime.datetime.now().strftime("%Y%m%dT%H%M%S"))

In [ ]:
start_time = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")

nbatches = 4
ntrials_set = np.array([2, 4, 8, 16])
nfix_set = np.array([0,1,2,3,4,5])

Parallel(n_jobs=num_cores, verbose=5)(
    delayed(fit_GP)(nfix, ntrials, batchnum, start_time) for ntrials, nfix, batchnum in itertools.product(ntrials_set, nfix_set, range(nbatches))
)

# start_time = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")

# nbatches = 4 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])

# Parallel(n_jobs=num_cores, verbose=5)(
#     delayed(fit_GP)(nfix, ntrials, batchnum, start_time) for ntrials, nfix, batchnum in itertools.product(ntrials_set, nfix_set, range(nbatches))
# )

# nbatches = 2
# ntrials_set = np.array([8, 16])
# nfix_set = np.array([0,1,2,3,4,5])

# Parallel(n_jobs=num_cores)(
#     delayed(fit_GP)(nfix, ntrials, batchnum, start_time) for ntrials, nfix, batchnum in itertools.product(ntrials_set, nfix_set, range(nbatches))
# )

# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])

# Parallel(n_jobs=num_cores)(
#     delayed(fit_GP)(nfix, ntrials, batchnum, start_time) for ntrials, nfix, batchnum in itertools.product(ntrials_set, nfix_set, range(nbatches))
# )


# nbatches = 1 
# ntrials_set = np.array([128])
# nfix_set = np.array([0,1,2,3,4])

# Parallel(n_jobs=num_cores)(
#     delayed(fit_GP)(nfix, ntrials, batchnum, start_time) for ntrials, nfix, batchnum in itertools.product(ntrials_set, nfix_set, range(nbatches))
# )


## 3. Load and compare fits

In [6]:
start_time = '20180204T121137'
nbatches = 4
ntrials_set = np.array([2, 4, 8, 16])
nfix_set = np.array([0,1,2,3,4,5])


# start_time = '20180119T185848'
# nbatches = 4 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])


# start_time = '20180119T121802'
# nbatches = 2
# ntrials_set = np.array([8, 16])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180118T224743'
# nbatches = 2
# ntrials_set = np.array([8, 128])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180118T174510'
# nbatches = 2
# ntrials_set = np.array([2, 4, 8])
# nfix_set = np.array([0,2,4,5])


# start_time = '20180118T174510'
# nbatches = 2
# ntrials_set = np.array([2, 4, 8])
# nfix_set = np.array([0,2,4,5])

# start_time = '20180117T191020'
# nbatches = 10
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180117T032858'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])


# start_time = '20180105T145529'
# nbatches = 1
# ntrials_set = np.array([128])
# nfix_set = np.array([0,1,2,3,4])


# start_time = '20180105T145529'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4])

# start_time = '20180104T170845'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4])

In [7]:
# Establish test dataset
y_test = y[:,:,350:]
x_test = x[:,:,350:]

In [8]:
# Load the saved files
def load_GP_fit(ntrials_ind, nfix_ind, batchnum_ind, ntrials, nfix, batchnum, start_time):
    load_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (nfix, ntrials, batchnum)
    load_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + load_fname_params + ".pkl"
    # load_fname_params = "_%2d_fix_%3d_trials" % (nfix, ntrials)
    # load_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + load_fname_params + ".pkl"

    with open(load_fname, 'r') as f:
        # Load and store results
        results_file = pickle.load(f)
    
    return results_file

GP_fit_saves = Parallel(n_jobs=num_cores)(
    delayed(load_GP_fit)(ntrials_ind, nfix_ind, batchnum_ind, ntrials, nfix, batchnum, start_time)
    for (ntrials_ind, nfix_ind, batchnum_ind), (ntrials, nfix, batchnum) in (
        itertools.izip(
            itertools.product(range(len(ntrials_set)), range(len(nfix_set)), range(nbatches)),
            itertools.product(ntrials_set, nfix_set, range(nbatches))
        )
    )
)

In [9]:
# Create predictions for each test set trial
def test_GP_pred(y_test, GPfit, cutoff=None):
    # Sort results into variables to work with
    [y_train, x_train, kwells_params,
                 all_results, 
                 init_paramvec, dict_ind, dict_shape, opt_params, 
                 bnds, transforms] = GPfit                                   
                                   
                                   
    # Get GP predictions on test data
    return pred_GP(y_test, 
                   replace_params(all_results[-1].x, opt_params, init_paramvec), 
                   transforms, dict_ind, dict_shape, cutoff = cutoff)


# Load or make and save predictions
cutoff = None
preds_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + "_predictions.pkl"
try:
    [GP_predictions, ntrials_set, nfix_set, nbatches] = (
        pickle.load(open(preds_fname, 'rb')))
except (OSError, IOError) as e:
    GP_predictions = Parallel(n_jobs=num_cores)(
        delayed(test_GP_pred)(y_test, GPfit, cutoff=cutoff)
        for GPfit in GP_fit_saves
    )
    pickle.dump([GP_predictions, ntrials_set, nfix_set, nbatches], 
                open(preds_fname, 'wb'))


In [10]:
# Get errors based on predictions
axisError = (0,1,2)

AR_RMSE = []
for GPfit in GP_fit_saves:
    AR_RMSE.append(rmse(pred_lin_AR1(y_test, GPfit[0], cutoff=cutoff), y_test, axis=axisError))

GP_NLL = []
GP_RMSE = []
for GPpred in GP_predictions:
    GP_NLL.append(np.sum(GPpred[5], axis=tuple(np.array(axisError)[1:]-1)))
    GP_RMSE.append(rmse(GPpred[7], y_test, axis=axisError))
    

AR_RMSE = np.reshape(np.array(AR_RMSE), (len(ntrials_set), len(nfix_set), nbatches))
GP_RMSE = np.reshape(np.array(GP_RMSE), (len(ntrials_set), len(nfix_set), nbatches))
GP_NLL = np.reshape(np.array(GP_NLL), (len(ntrials_set), len(nfix_set), nbatches))


In [11]:
np.set_printoptions(precision=2)
print "AR_RMSE"
print np.array(np.median(AR_RMSE, axis=2))
print "-------------"
print "GP_RMSE"
print np.array(np.median(GP_RMSE, axis=2))
print "-------------"
print "GP_NLL"
print np.array(np.median(GP_NLL, axis=2))

AR_RMSE
[[ 0.37  0.37  0.37  0.37  0.37  0.37]
 [ 0.37  0.37  0.37  0.37  0.37  0.37]
 [ 0.37  0.37  0.37  0.37  0.37  0.37]
 [ 0.37  0.37  0.37  0.37  0.37  0.37]]
-------------
GP_RMSE
[[ 0.29  0.54  0.35  0.37  0.32  0.33]
 [ 0.27  0.27  0.27  0.27  0.27  0.27]
 [ 0.26  0.27  0.27  0.27  0.27  0.27]
 [ 0.26  0.26  0.26  0.26  0.26  0.26]]
-------------
GP_NLL
[[ 1245.69  4321.33  4074.59  3765.25  2622.38  4830.39]
 [  493.7    581.44   652.09   606.64   446.34   428.46]
 [  209.88   260.35   169.96   194.57   226.16   290.08]
 [  172.28   141.88   116.54   111.33   167.68   154.2 ]]


In [12]:
# Plot negative log likelihoods for each options
def plot_nll_fixpoints(cur_nll_ntrial, nfix_set, nbatches):
    cur_plts = []
    for (nfix_ind, nfix) in itertools.izip(range(len(nfix_set)), nfix_set):
        cur_plts.append(plt_type.Scatter(x=nfix*np.ones_like(np.squeeze(cur_nll_ntrial[nfix_ind,:])), 
                                         y=np.squeeze(cur_nll_ntrial[nfix_ind,:]), 
                                         mode='markers', 
                                         marker = dict(color=plotly.colors.DEFAULT_PLOTLY_COLORS[nfix_ind]),
                                         name = "nfix=%d" % nfix, 
                                         legendgroup="nfix=%d" % nfix))
    return cur_plts
    
    
from plotly import tools as plt_tools

def plot_nll(cur_nll, ntrials_set, nfix_set, nbatches):
    fig_subplts = plt_tools.make_subplots(rows=1, cols=len(ntrials_set), subplot_titles=ntrials_set, shared_yaxes=True)
    for (ntrials_ind, ntrials) in itertools.izip(range(len(ntrials_set)), ntrials_set):        
        tmp = plot_nll_fixpoints(cur_nll[ntrials_ind,:,:], nfix_set, nbatches)
        for trace in tmp:
            fig_subplts.append_trace(trace,1,ntrials_ind+1)
            if ntrials_ind>0:
                fig_subplts['data'][-1]['showlegend']=False
    plt(fig_subplts)
    return fig_subplts

a = plot_nll(GP_NLL, ntrials_set, nfix_set, nbatches)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]



In [13]:
np.set_printoptions(precision=2)
print "-------------"
print "GP_NLL"
print np.array(np.min(GP_NLL, axis=2))
print "-------------"
print "GP_NLL"
print np.array(np.argmin(GP_NLL, axis=2))

-------------
GP_NLL
[[  675.95  2989.87  2988.51  2793.88  1194.25  1738.28]
 [  155.55   332.37   240.23   200.44   235.4    173.81]
 [  193.05   179.48   142.27   165.72   151.67   228.11]
 [  114.08    64.25    67.      35.87   101.34    91.24]]
-------------
GP_NLL
[[1 3 0 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 3 3 3]
 [3 3 2 3 3 3]]


In [14]:
def nestedlist_to_array(nested_list, max_level = 0):
    cur_nested_list = nested_list
    level = 0
    array_dims = []
    array_dims_range = []
    while (level <= max_level) and (type(cur_nested_list)==list):
        array_dims.append(len(cur_nested_list))
        array_dims_range.append(range(len(cur_nested_list)))
        cur_nested_list = cur_nested_list[0]
        level += 1

    out = np.empty(tuple(array_dims), dtype=object)
    
    for inds in itertools.product(*array_dims_range):
        cur_nested_list = nested_list
        for i in inds:
            cur_nested_list = cur_nested_list[i]
        out[inds] = cur_nested_list
    
    return out

In [28]:
# Plot a single fit
GP_fit_saves_rs = np.reshape(nestedlist_to_array(GP_fit_saves, max_level=0), 
                             (len(ntrials_set), len(nfix_set), nbatches))

nfix_ind = 5
ntrials_ind = 3
batchnum = 0

print GP_RMSE[ntrials_ind, nfix_ind, batchnum]
print GP_NLL[ntrials_ind, nfix_ind, batchnum]

[y_train, x_train, kwells_params,
 all_results, 
 init_paramvec, dict_ind, dict_shape, opt_params, 
 bnds, transforms] = GP_fit_saves_rs[ntrials_ind, nfix_ind, batchnum]

print len(init_paramvec)

kwells_callback_plot_external(init_paramvec[opt_params], 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        kwells_params)

# for res in all_results[:-1]:
#     kwells_callback_plot_external(res.x, 
#             opt_params, init_paramvec, transforms, dict_ind, dict_shape,
#             kwells_params)
    
kwells_callback_plot_external(all_results[-1].x, 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        kwells_params)

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()
    
print np.concatenate([s.T, Sigma_s], axis=1)

0.270853383653
195.595205804
60


[[  9.12483930e-03   4.15722102e-07]
 [ -9.54580950e-01   4.04308110e-03]
 [  1.02358967e+00   2.56580791e-04]
 [  4.63581493e-01   3.35060329e-02]
 [ -3.97045403e-01   4.38201735e-02]]


In [24]:
paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()

In [ ]:
len(all_results)

In [25]:
np.set_printoptions(precision=8)
paramdict

OrderedDict([('Sigma_eps', array([[ 0.01223869]])),
             ('mu_0_0', array([[-0.00224997]])),
             ('Sigma_0_0', array([[  1.08350763e-14]])),
             ('C', array([[ 1.]])),
             ('Sigma_nu', array([[ 0.001]])),
             ('z',
              array([[-1.23793413, -1.2371736 , -1.23743931, -0.60703914, -0.60551151,
                      -0.32193317, -0.31968079,  0.01796561,  0.01695618,  0.49670735,
                       0.49810686,  0.76195272,  1.01255883]])),
             ('u',
              array([[-0.76653048, -0.76832069, -0.76713589, -0.77997034, -0.77866505,
                       0.14489087,  0.15471138,  0.07634256,  0.07559478,  0.64724952,
                       0.70798009,  0.24778277,  0.81673731]])),
             ('Sigma_u', array([[  2.71677521e-06],
                     [  3.45630724e-06],
                     [  2.02071456e-07],
                     [  1.44281394e-03],
                     [  1.16431031e-03],
                     [  1.37

In [ ]:
vec_to_params(init_paramvec, dict_ind, dict_shape, transforms)

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u
print s
print Sigma_s
print Sigma_J

In [ ]:
print lengthscales

In [ ]:
# Add priors (to span at least the bounds)
priors = []

# Add prior to ensure inducing points are smooth
cur_prior = {}
cur_prior['func'] = create_prior(prior_distribution="InducingSmooth")
cur_prior['inds'] = range(len(init_paramvec))
cur_prior['metadata'] = {}
def unpack_x_tmp(x):
    # Unpack all the usual parameters
    param_tuple = vec_to_params(x, dict_ind, dict_shape)
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, 
     z, u, Sigma_u, 
     lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
        param_tuple

    kernelparams = {'lengthscales': lengthscales, 'kernel_variance': kernel_variance}
    # Return the ones we want in the required format
    return (np.concatenate([u, s],axis=1), 
            np.concatenate([z, s],axis=1),
            np.concatenate([Sigma_u, Sigma_s]),
            kernelparams)
cur_prior['metadata']['unpack_x'] = unpack_x_tmp
cur_prior['metadata']['kernel_func'] = RBF
priors.append(cur_prior)

tmp_func = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                                y_train, dict_ind, dict_shape, 
                                                log_transformed=log_transformed,
                                                priors=priors)[0])


In [ ]:
paramvec = replace_params(all_results[1].x, opt_params, init_paramvec)
#paramvec = log_transform_inv(paramvec, log_transformed)
paramvec = paramvec[opt_params]
a = grad(tmp_func)(paramvec)

In [ ]:
paramvec = replace_params(a, opt_params, init_paramvec)
#paramvec = log_transform_inv(paramvec, log_transformed)

# Unpack the usual parameters
param_tuple = vec_to_params(paramvec, dict_ind, dict_shape)
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    param_tuple

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u
print s
print Sigma_s

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u

In [ ]:
print s
print Sigma_s

In [ ]:
vec_to_params(init_paramvec, dict_ind, dict_shape, transforms=transforms)